# Split data by Stadsdeel

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import shapely.geometry as sg

In [ ]:
# Load neighbourhood data and convert to RD
neighbourhood_data = '../datasets/neighbourhoods_amsterdam.csv'

neighbourhood_gdf = pd.read_csv(neighbourhood_data, index_col=0)
neighbourhood_gdf['geometry'] = gpd.GeoSeries.from_wkt(neighbourhood_gdf['geometry'])
neighbourhood_gdf = gpd.GeoDataFrame(neighbourhood_gdf, crs='wgs84')
neighbourhood_gdf.to_crs(epsg='28992', inplace=True)

In [ ]:
district_dict = {'B': 'haven',
                 'T': 'zuid_oost',
                 'A': 'centrum',
                 'K': 'zuid',
                 'E': 'west',
                 'M': 'oost',
                 'N': 'noord',
                 'F': 'nieuw_west'}

In [ ]:
neighbourhood_gdf['naam'] = neighbourhood_gdf['district_code'].replace(district_dict)
stadsdelen_poly = neighbourhood_gdf[['naam', 'geometry']].dissolve(by='naam')

In [ ]:
stadsdelen_poly.to_file('stadsdelen.gpkg', driver='GPKG')

In [ ]:
# Load tile data
tiles_file = f'../datasets/tile_info/AMS_run1_tiles_list.csv'
all_tiles_gdf = pd.read_csv(tiles_file)
all_tiles_gdf['geometry'] = all_tiles_gdf.apply(
    lambda row: sg.box(row.RD_X, row.RD_Y, row.RD_X+50, row.RD_Y+50), axis=1)
all_tiles_gdf = gpd.GeoDataFrame(all_tiles_gdf)

In [ ]:
drop_folders = ('nl-amsd-200605-7415-laz', 'nl-amsd-200625-7415-laz')
all_tiles_gdf = all_tiles_gdf[~all_tiles_gdf['folder'].isin(drop_folders)]

In [ ]:
folder_dict = {'nl-amsd-200519-7415-laz': 'haven',
               'nl-amsd-200604-7415-laz': 'zuid_oost',
               'nl-amsd-200606-7415-laz': 'centrum',
               'nl-amsd-200608-7415-laz': 'zuid',
               'nl-amsd-200618-7415-laz': 'west',
               'nl-amsd-200622-7415-laz': 'oost',
               'nl-amsd-200702-7415-laz': 'noord',
               'nl-amsd-200707-7415-laz': 'nieuw_west'}

In [ ]:
all_tiles_gdf['naam'] = all_tiles_gdf['folder'].replace(folder_dict)

In [ ]:
stadsdelen_tiles = all_tiles_gdf[['naam', 'geometry']].dissolve(by='naam')

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

stadsdelen_tiles.plot(ax=ax, cmap='tab10', alpha=0.5, legend=True)
stadsdelen_poly.boundary.plot(ax=ax, edgecolor='black')
# stadsdelen_poly.plot(ax=ax, column='district_code', cmap='tab10', legend=True)

In [ ]:
plt.savefig('stadsdelen_dekking.png', ax=ax, dpi=200)